In [ ]:
# TODO: author email ending removal 
# 

In [2]:
import os, json
from tqdm import tqdm
from pymilvus import connections,Collection, utility, FieldSchema, CollectionSchema, DataType
from dotenv import load_dotenv
import torch
from sentence_transformers import SentenceTransformer
from langchain_text_splitters import RecursiveCharacterTextSplitter

device = "cuda" if torch.cuda.is_available() else "cpu"
# for big model BAAI/bge-m3
model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1", device=device)
BATCH_SIZE = 100
load_dotenv()


True

In [3]:
print(device)
print(torch.version.cuda)

cuda
11.8


In [4]:

connections.connect(
    alias="default",
    host="127.0.0.1",
    port="19530",
    timeout=5,
    secure=False
)

print("Connected to Milvus:", utility.get_server_version())


Connected to Milvus: v2.3.21


In [5]:

# incase I need to delete it
collection_name = "osdr"

# Check if collection exists
if utility.has_collection(collection_name):
    utility.drop_collection(collection_name)
    print(f"Collection '{collection_name}' deleted.")
else:
    print(f"Collection '{collection_name}' does not exist.")

Collection 'osdr' does not exist.


In [8]:
def load_docs(folder_path:str) -> list:
    """load json files from specified folder into list, not load if text field is empty"""
    docs = list()

    for file in os.listdir(folder_path):
        if file.endswith(".json"):
            with open(os.path.join(folder_path, file) , "r", encoding="utf-8") as doc:
                data = json.load(doc)
                if data["description"] != "":
                    docs.append(data)
                
    return docs

In [9]:

folder_path = "./data/osdr_raw/"
documents = load_docs(folder_path)


In [14]:
def clean_osd_name(osd:json):
    """clean osd name from trailing escape characters and spaces"""
    osd["study_name"] = osd["study_name"].replace(r"\n","").strip()
def clean_text(osd:json):
    """strip main text"""
    osd["text"] = osd["text"].strip()

def remove_email(osd:json):
    if len(osd["authors"]) > 1:
        authors_cleaned = []
    
        for author in osd["authors"]:
            if author.endswith("email"):
                author = author.replace("email","")
            author = author.strip()
            authors_cleaned.append(author)
        osd["authors"] = authors_cleaned



In [15]:
for doc in documents:
    clean_osd_name(doc)
    remove_email(doc)